In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy

# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

from utils.calcium import calcium

from utils.wheel import wheel

Autosaving every 180 seconds


<ipython-input-1-3df9a861b721>:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


/home/cat/.conda/envs/donato/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#########################################################
########## COMPUTE VELOCITY FROM ROTARY ENCODER ##########
##########################################################
from tqdm import trange

#
def binarize(data):
    
    idx = np.where(data<2.5)[0]
    idx2 = np.where(data>=2.5)[0]
    
    data[idx] = 0
    data[idx2] = 1
    
    #
    return data

#
def get_velocity(rot1, rot2):
    
    # distance
    n_clicks_per_rotation = 500
    ball_diameter = 0.2  # distance in meters
    ball_circumference = ball_diameter*3.141592
    dist_per_click = ball_circumference/n_clicks_per_rotation
    
    # time
    sample_rate = 1000
    seconds_per_time_stamp = 1/sample_rate
    
    
    #
    bin1 = binarize(rot1)
    bin2 = binarize(rot2)
    
    #
    clicks = np.array((bin1, bin2)).T.squeeze()
    print (clicks)
        
    #
    vel = []
    times = []
    last_click_location = 0
    time_since_last_click = 0
    rot1_last_state = clicks[0,0]
    rot2_state = clicks[0,1]
    for k in trange(0,clicks.shape[0],1):
        
        #
        if clicks[k,0]!=rot1_last_state:
            distance = dist_per_click  #only walked 1 click
            time = time_since_last_click*seconds_per_time_stamp
            
            # 
            v = distance/time 
            
            #
            vel.append(v)
            times.append(k/sample_rate)       
            
            #
            time_since_last_click=0
            last_click_location=k #.copy()
            rot1_last_state=clicks[k,0]
        else:
            vel.append(0)
            times.append(k/sample_rate)       

        #    
        time_since_last_click+=1
    #
    return np.array(vel), np.array(times)
    

In [ ]:
#
# import scipy
# fname_mat = '/media/cat/4TB1/donato/DON-009191/20220220/TRD-2P/DON-009191_20220220_TRD-2P_S1-ACQ.mat'

# data = scipy.io.loadmat(fname_mat)
# #print (data)

# print (data['trainingdata'].shape)
# rot1 = data['trainingdata'][:,0]
# rot2 = data['trainingdata'][:,1]

# #
# vel, times = get_velocity(rot1,rot2)

root_names = '/home/cat/Downloads/trd_matfile_names.txt'


with open(root_names) as f:
    fnames = f.readlines()


print (fnames[0])    
#
for fname in fnames:
    
    idx = fname.find("TRD-2P")
    #print (idx)
                     
    fname1 = fname[:idx+7]+"wheel.npy"
    
    
    #
    fname1 = fname1.replace('\\', '/')
    
    print ("fname 1: ", fname1)
   # break
    #
    try:
        data_wheel = np.load(fname1)
    except:
        print (" error processing file: ", fname1)
        continue


    w = wheel.Wheel()
    w.root_dir = os.path.split(fname1)[0]
    w.load_track()
    w.compute_velocity()
                               
                            
    #break


/run/user/1000/gvfs/smb-share:server=scicore-toucan.scicore.unibas.ch,share=donafl00-calcium$/Users/Rodrigo/DON-006084/20210519/TRD-2P/DON-006084_20210519_TRD-2P_S1-ACQ.mat

fname 1:  /run/user/1000/gvfs/smb-share:server=scicore-toucan.scicore.unibas.ch,share=donafl00-calcium$/Users/Rodrigo/DON-006084/20210519/TRD-2P/wheel.npy
fname 1:  /run/user/1000/gvfs/smb-share:server=scicore-toucan.scicore.unibas.ch,share=donafl00-calcium$/Users/Rodrigo/DON-006084/20210520/TRD-2P/wheel.npy


100%|█████████████████████████████| 2787808/2787808 [00:11<00:00, 235459.75it/s]


non-zero rotary encoder vals:  [      17      116      211 ... 18297307 18297605 18298432]


100%|█████████████████████████████████| 28578/28578 [00:00<00:00, 110591.30it/s]


vel_times:  [      17      115      211 ... 18297307 18297605 18298432]


...

non-zero rotary encoder vals:  [      31      166      303 ... 18293214 18293855 18294465]


100%|███████████████████████████████████| 8950/8950 [00:00<00:00, 103189.56it/s]


vel_times:  [      31      166      303 ... 18293214 18293855 18294465]


refilling stationary times: 100%|███████| 8949/8949 [00:00<00:00, 169918.50it/s]


vel times:  (11496,)
vel values:  (11496,) [0.40536681 0.09308422 0.09172533 ... 0.00575647 0.01960432 0.02060061]
self.track.galvo_triggers.times:  [  140285   140607   140930 ... 18147462 18147785 18148108]
vel_times: [      31      166      303 ... 18293214 18293855 18294465]


In [10]:
####################################################
def get_h5py_speed(filename):
    import h5py
    #filename = "file.hdf5"

    with h5py.File(filename, "r") as f:
        # Print all root level object names (aka keys) 
        # these can be group or dataset names 
        print("Keys: %s" % f.keys())
        # get first object name/key; may or may NOT be a group
        a_group_key = list(f.keys())[1]

        # get the object type for a_group_key: usually group or dataset
        print(type(f[a_group_key])) 

        # If a_group_key is a group name, 
        # this gets the object names in the group and returns as a list
        data = list(f[a_group_key])

        # If a_group_key is a dataset name, 
        # this gets the dataset values and returns as a list
        data = list(f[a_group_key])

        # preferred methods to get dataset values:
        ds_obj = f[a_group_key]      # returns as a h5py dataset object
        ds_arr = f[a_group_key][()].squeeze()  # returns as a numpy array

        #

    return ds_arr

################################################
vel_python = np.load('/media/cat/4TB1/donato/DON-009191/20220217/TRD-2P/velocity.npy')*100


filename = '/media/cat/4TB1/donato/DON-009191/20220217/TRD-2P/trd_var.mat'
vel_matlab = get_h5py_speed(filename)
####################################################

plt.figure()
#times = np.arange(vel_python.shape[0])/30.
plt.plot(w.track.velocity.times[start:end],
         w.track.velocity.values[start:end]*100, c='blue', label='python')

#
times_matlab = np.arange(vel_matlab.shape[0])/10000.
plt.plot(times_matlab[::30], vel_matlab[::30], c='red',label='matlab')
plt.legend(fontsize=16)
plt.show()

Keys: <KeysViewHDF5 ['raw_dist', 'spd', 'tt']>
<class 'h5py._hl.dataset.Dataset'>


In [5]:
##################################################################
############# COMPUTE VELOCITY FROM ROTARY ENCODERS ##############
##################################################################
data = np.load('/home/cat/Downloads/Downloads/results.npz',
               allow_pickle=True)

#
lick_detector_times = data['lick_detector_abstime']
print ("lick detector: ", lick_detector_times.shape, lick_detector_times)

#
reward_times = data['reward_times'].squeeze().T[:,1]
#print ("reward times: ", reward_times[:50])
ttl_times = data['ttl_times']
#print ("ttl times: ", ttl_times)

#
rot1 = data['rotary_encoder1_abstime']
rot2 = data['rotary_encoder2_abstime']
#print ("rotary 1: ", rot1)
#print ("rotary 2: ", rot2)

#
vel, times = get_velocity(rot1,rot2)

plt.figure()
plt.plot(times, vel)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/home/cat/Downloads/Downloads/results.npz'

In [21]:
#
vel2 = np.load('/media/cat/4TB1/donato/DON-009191/20220217/TRD-2P/wheel.npy').squeeze()
print (vel.shape, vel2.shape)

plt.figure()
#plt.plot(times, vel, c='blue')
plt.plot(times, vel2, c='red',)
plt.show()

(18300000,) (18300000,)


In [67]:
#################################################
#################### PUP IDS ####################
#################################################

#
mouse_ids = [
    #'006084',
    #'006085',
    #'006087',
    '009191',
    '009192',
    '010473',
    '010477'
]

#
PDays = [
    #[19,28],
    #[19,28],
    #[24,33],
    [16,48],  # expression issues
    [15,60],  
    [16,33],
    [15,33],
]

#
fig=plt.figure()
ax=plt.subplot()
xlabels = mouse_ids

#
for k in range(len(mouse_ids)):
    #y = np.arange(PDays[k][0],PDays[k][1],0.1)
    x = np.arange(PDays[k][0],PDays[k][1],0.1)
    y = k
    #plt.fill_betweenx(y, k, k+0.9, 
                      #alpha=.5, label = mouse_ids[k])
    
    #x = np.arange(
    ax.fill_between(x,k,k+0.9,
                   label = mouse_ids[k])
    
    plt.text(np.median(x)-1,k+0.3,str(PDays[k][1]-PDays[k][0]+1))

#
plt.xlim(left=0)
plt.ylabel("mice ids")
plt.yticks(np.arange(len(mouse_ids))+0.5, 
           mouse_ids, 
           rotation=45)
plt.xlabel("P-Day")
plt.legend(ncol=2, fontsize=8, loc=4)
plt.show()




In [5]:
#################################################
################## ADULT IDS ####################
#################################################

#
mouse_ids = [
    '002865', 
    '003165',
    '003343',
    '008497',
    '008498',
    '008499'
]

#
PDays = [
    [90,99],
    [120,129],
    [150,159],
    [61,73],
    [61,73],
    [61,73],
]

#
fig=plt.figure()
ax=plt.subplot()
xlabels = mouse_ids

for k in range(len(mouse_ids)):
    #y = np.arange(PDays[k][0],PDays[k][1],0.1)
    x = np.arange(PDays[k][0],PDays[k][1],0.1)
    y = k
    #plt.fill_betweenx(y, k, k+0.9, 
                      #alpha=.5, label = mouse_ids[k])
    
    #x = np.arange(
    alpha=1.0
    if k >2:
        alpha=.3
    ax.fill_between(x,k,k+0.9, label = mouse_ids[k],
                   alpha=alpha)
    
    plt.text(np.median(x)-3,k+0.3,str(PDays[k][1]-PDays[k][0]+1))

plt.xlim(left=0)
plt.ylabel("mice ids")
plt.yticks(np.arange(len(mouse_ids))+0.5, mouse_ids,
          rotation=45)
plt.xlabel("P-Day")
plt.legend(ncol=2, fontsize=8, loc=4)
plt.show()


(18299999,)


(18299999,)
